In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import math

ASSETS_FOLDER = 'assets/'

In [ ]:
def prepare_image():
    print("prepare")

In [ ]:
def load_images_from_assets():
  images = []
  for filename in os.listdir(ASSETS_FOLDER):
      image = cv2.imread(os.path.join(ASSETS_FOLDER, filename), -1)
      if image is not None:
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          images.append(image)

  return images

In [ ]:
def display_images(images, num, f1, f2, title = ""):
    indices = np.random.default_rng().choice(len(images), num, replace=False)
    _, axes = plt.subplots(f1, f2, figsize=(8, 3), squeeze=False) 
    for r in range(axes.shape[0]):
        for c in range(axes.shape[1]):
            axes[r, c].imshow(images[indices[r * axes.shape[1] + c]])
            axes[r, c].axis("off")
    plt.title(title)
    plt.tight_layout()

In [ ]:
def display_dataset_info(images, title, num, f1, f2):
    heights = list(map(lambda image: image.shape[0], images))
    widths = list(map(lambda image: image.shape[1], images))

    print(f"Število slik: {len(images)}")
    print(f"Interval širine slik: [{np.min(widths)}, {np.max(widths)}]") 
    print(f"Interval višine slik: [{np.min(heights)}, {np.max(heights)}]") 
    print(f"Povprečna širina slik: {np.mean(widths) :.2f} +- {np.std(widths) :.2f}") 
    print(f"Povprečna višina slik: {np.mean(heights) :.2f} +- {np.std(heights) :.2f}") 
    print("Primeri slik:")
    display_images(images, num, f1, f2, title)

In [ ]:
def preprocess(images, factor, type):
    low_res = []
    high_res = []

    for image in images:
        smaller_size = (int(image.shape[1] / factor), int(image.shape[0] / factor))
        bigger_size = (int(smaller_size[0] * factor), int(smaller_size[1] * factor))
        low_res_image = cv2.resize(image, smaller_size, cv2.INTER_LINEAR)
        high_res_image = cv2.resize(low_res_image, bigger_size, cv2.INTER_LINEAR)

        low_res_image = cv2.cvtColor(low_res_image, cv2.COLOR_BGR2YCR_CB)
        high_res_image = cv2.cvtColor(high_res_image, cv2.COLOR_BGR2YCR_CB)

        if type == 0:
            low_res_image = cv2.copyMakeBorder(low_res_image, 0, 33, 0, 33, cv2.BORDER_CONSTANT, None, value = 0)
            high_res_image = cv2.copyMakeBorder(high_res_image, 0, 33, 0, 33, cv2.BORDER_CONSTANT, None, value = 0)
            
            for i in range(0, smaller_size[0], 14):
                for j in range(0, smaller_size[1], 14):
                    low_res.append(low_res_image[j:j+33, i:i+33])


            for i in range(0, bigger_size[0], 14):
                for j in range(0, bigger_size[1], 14):
                    high_res.append(high_res_image[j:j+33, i:i+33])
        else:
            low_res.append(low_res_image[:,:,0])
            high_res.append(high_res_image[:,:,0])

    display_dataset_info(low_res, "LOW RESOLUTION IMAGES, FACTOR: " + str(factor), 15, 3, 5)
    display_dataset_info(high_res, "HIGH RESOLUTION IMAGES, FACTOR: " + str(factor), 15, 3, 5)
    return low_res, high_res

In [ ]:
images = load_images_from_assets()

display_dataset_info(images, "DATASET", 2, 1, 2)

In [ ]:
factor = 2
low_res, high_res = preprocess(images, factor, 0)